## Objective of the Case Study
How many ordered items will be exported to a specific warehouse (warehouse 2) on a daily basis for each country based on the sales forecast.

To achieve this we will accumulate the item values from different types of ordering processess, all of them have different export rules.

## Imports

In [2]:
import pandas as pd

## Fetch data

In [3]:
# Here we are reading the csv files
b2c_sales_df = pd.read_csv('/Users/nitinnandansingh/Documents/workspace/AboutYouLogisticsTask/assets/b2c_items_sales_forecast - b2c_items_sales_forecast.csv')
scd_sales_df = pd.read_csv('/Users/nitinnandansingh/Documents/workspace/AboutYouLogisticsTask/assets/scd_items_sales_foreecast - scd_items_sales_forecast.csv')
icd_sales_df = pd.read_csv('/Users/nitinnandansingh/Documents/workspace/AboutYouLogisticsTask/assets/icd_items_sales_forecast - icd_items_sales_forecast.csv')
it_cd_shift_df = pd.read_csv('/Users/nitinnandansingh/Documents/workspace/AboutYouLogisticsTask/assets/item_cross_docking_shift - item_cross_docking_shift.csv')
it_blocked_df = pd.read_csv('/Users/nitinnandansingh/Documents/workspace/AboutYouLogisticsTask/assets/items_blocked_per_cd_item - items_blocked_per_cd_item.csv')

In [4]:
# i will first convert date values into datetime type. 
# This information I gathered after performing some analyses beforehand. 
# Please refer to the notebook called explore_assets for detailed exploration of the datasets.

b2c_sales_df['date'] = pd.to_datetime(b2c_sales_df['date'])
b2c_sales_df['forecast_date'] = pd.to_datetime(b2c_sales_df['forecast_date'])

scd_sales_df['date'] = pd.to_datetime(scd_sales_df['date'])
scd_sales_df['forecast_date'] = pd.to_datetime(scd_sales_df['forecast_date'])


icd_sales_df['date'] = pd.to_datetime(icd_sales_df['date'])
icd_sales_df['forecast_date'] = pd.to_datetime(icd_sales_df['forecast_date'])


it_cd_shift_df['exported_at_picking_warehouse'] = pd.to_datetime(it_cd_shift_df['exported_at_picking_warehouse'])


it_blocked_df['date'] = pd.to_datetime(it_blocked_df['date'])

### Let's look at the data

The following set of cells shows first 3 rows of the dataframes we created from the csv files.

In [5]:
b2c_sales_df.head(3)

,forecast_date,version,scenario_id,date,planning_shop_cluster,shipping_wh,sales_forecast_items
0,2024-06-04,1,0,2024-05-27,AT,1,2277
1,2024-06-04,1,0,2024-05-27,AT,2,0
2,2024-06-04,1,0,2024-05-27,AT,4,0


In [6]:
scd_sales_df.head(3)

,forecast_date,version,scenario_id,date,planning_shop_cluster,shipping_wh,picking_wh,shipment_cd
0,2024-06-04,1,0,2024-07-22,CZ,3,3,0
1,2024-06-04,1,0,2024-05-31,HU,3,3,0
2,2024-06-04,1,0,2024-08-24,NO,3,3,0


In [7]:
icd_sales_df.head(3)

,forecast_date,version,scenario_id,date,planning_shop_cluster,shipping_wh,picking_wh,item_cd
0,2024-06-04,1,0,2024-07-22,CZ,3,3,0
1,2024-06-04,1,0,2024-05-31,HU,3,3,0
2,2024-06-04,1,0,2024-08-24,NO,3,3,0


In [8]:
it_cd_shift_df.head(3)

,exported_at_picking_warehouse,picking_warehouse_id,shipping_warehouse_id,day,share
0,2024-05-27,1,2,0,0.02
1,2024-05-27,1,2,1,0.23
2,2024-05-27,1,2,2,0.61


In [9]:
it_blocked_df.head(3)

,date,shipping_wh,country_code,items_per_cross_docking_cd_item
0,2023-03-01,1,AT,3.17
1,2023-03-01,2,AT,3.17
2,2023-03-01,3,AT,3.17


## Tasks Implementation

### Step 1

Based on export rules, aggregate B2C (shipping warehouse) and SCD (picking warehouse) on exported_at for warehouse_id=2

In this step, first I will select b2c orders where shipping warehouse number is 2. In such orders, the created_at date is same as exported_at date. This is represented by date column in the dataframe.

Then, I will select those scd orders, for which shipping warehouse is 2 but picking warehouse is not 2. This is because, we want to find out how many items are to be exported to warehouse 2, then warehouse 2 is the default/shipping warehouse but no item will be picking from there (i.e warehouse 2 is not a picking warehouse). Instead, all items will be exported to shipping warehouse 2 from non-default warehouses.

In [10]:
# Here first I filter the b2c sales data with shipping warehouse number 2 and then group them on date and planning_shop_cluster while summing up the sales_forecast_items
b2c_filtered = b2c_sales_df[b2c_sales_df['shipping_wh'] == 2]
b2c_aggregated = b2c_filtered.groupby(['date', 'planning_shop_cluster'], as_index=False)['sales_forecast_items'].sum()

# For my ease, i will rename the columns. The date column in this table is same as exported_at date as in regular b2c order date created = date exported.
b2c_aggregated.rename(columns={'date': 'exported_at', 'sales_forecast_items': 'b2c_items'}, inplace=True)
print(b2c_aggregated.shape)
b2c_aggregated.head()

(2639, 3)


,exported_at,planning_shop_cluster,b2c_items
0,2024-05-27,AT,0
1,2024-05-27,BE,0
2,2024-05-27,BG,754
3,2024-05-27,CH,0
4,2024-05-27,CY,34


In [11]:
# Here I am filtering the data from icd_sales dataframe based on shipping warehouse = 2 and picking warehouse =! 2 and then grouping them on date and planning_shop_cluster.
scd_filtered = scd_sales_df[(scd_sales_df['shipping_wh'] == 2) & (scd_sales_df['picking_wh'] != 2)]
scd_aggregated = scd_filtered.groupby(['date', 'planning_shop_cluster'], as_index=False)['shipment_cd'].sum()

# Here also I am renaming the columns and date created = date exported, in the case of scd.
scd_aggregated.rename(columns={'date': 'exported_at', 'shipment_cd': 'scd_items'}, inplace=True)
print(scd_aggregated.shape)
scd_aggregated.head()

(2639, 3)


,exported_at,planning_shop_cluster,scd_items
0,2024-05-27,AT,0
1,2024-05-27,BE,0
2,2024-05-27,BG,181
3,2024-05-27,CH,0
4,2024-05-27,CY,14


### Step 2

Aggregate ICD for shipping_warehouse_id=2 - this is the amount of items we forecast which will be needed to be sourced from other warehouses to fulfil the B2C orders in warehouse_id=2

In ICD orders, some items which are missing at default warehouse are picked from other warehouses and exported to default warehouses and consolidated before being shipped to the customer. So, to find out how many icd items are exported to shipping warehouse 2, we will filter out those orders for which shipping warehouse = 2.

In [12]:
# First we begin with filtering data for shipping warehouse 2.

icd_filtered = icd_sales_df[icd_sales_df['shipping_wh'] == 2]
print(icd_filtered.shape)
icd_filtered.head()


(10556, 8)


,forecast_date,version,scenario_id,date,planning_shop_cluster,shipping_wh,picking_wh,item_cd
13,2024-06-04,1,0,2024-06-07,GR,2,3,81
23,2024-06-04,1,0,2024-07-30,NL,2,4,0
32,2024-06-04,1,0,2024-06-04,ZZ,2,1,0
37,2024-06-04,1,0,2024-08-08,DE,2,2,0
38,2024-06-04,1,0,2024-08-11,ES,2,2,0


In [13]:
# We will group the data based on date and planning_shop_cluster and sum the item_cd. 
# This will give us a sum of items forecasted for icd for shipping warehouse 2 for each created_at date and country combination.
icd_filtered_aggregated = icd_filtered.groupby(['date', 'planning_shop_cluster'], as_index=False)['item_cd'].sum()

print(icd_filtered_aggregated.shape)
icd_filtered_aggregated.head()

(2639, 3)


,date,planning_shop_cluster,item_cd
0,2024-05-27,AT,0
1,2024-05-27,BE,0
2,2024-05-27,BG,103
3,2024-05-27,CH,0
4,2024-05-27,CY,9


### Step 3

Using results from point (B) of the and input (4), calculate how many items will be affected by item cross-docking process (items transported from picking warehouse + other items in the same order_id)  
-> results in dataframe  “items_minus”


input (4) : Assumption on how many items per ICD item are blocked from immediate export by the ICD process.


In this step, we want to find out how many items will be affected by cross-docking process. We have been told that for each icd item, more items are blocked. This value is given in the dataframe `it_blocked_df`'s `items_per_cross_docking_cd_item` column. I will merged it with `icd_filtered` df to calculate the total items.

In [14]:
merged_df = pd.merge(
    icd_filtered,
    it_blocked_df,
    left_on=['date', 'planning_shop_cluster', 'shipping_wh'],
    right_on=['date', 'country_code', 'shipping_wh'],
    how='inner'
)

print(merged_df.shape)
merged_df.head()

(10556, 10)


,forecast_date,version,scenario_id,date,planning_shop_cluster,shipping_wh,picking_wh,item_cd,country_code,items_per_cross_docking_cd_item
0,2024-06-04,1,0,2024-06-07,GR,2,3,81,GR,2.19
1,2024-06-04,1,0,2024-07-30,NL,2,4,0,NL,2.79
2,2024-06-04,1,0,2024-06-04,ZZ,2,1,0,ZZ,1.20
3,2024-06-04,1,0,2024-08-08,DE,2,2,0,DE,1.92
4,2024-06-04,1,0,2024-08-11,ES,2,2,0,ES,2.27


In [15]:
# Calculate the items affected by cross-docking by multiplying item_cd values with corresponding items_per_cross_docking_cd_item values.
merged_df['items_minus'] = merged_df['item_cd'] * merged_df['items_per_cross_docking_cd_item']
print(merged_df.shape)
merged_df.head()

(10556, 11)


,forecast_date,version,scenario_id,date,planning_shop_cluster,shipping_wh,picking_wh,item_cd,country_code,items_per_cross_docking_cd_item,items_minus
0,2024-06-04,1,0,2024-06-07,GR,2,3,81,GR,2.19,177.39
1,2024-06-04,1,0,2024-07-30,NL,2,4,0,NL,2.79,0.00
2,2024-06-04,1,0,2024-06-04,ZZ,2,1,0,ZZ,1.20,0.00
3,2024-06-04,1,0,2024-08-08,DE,2,2,0,DE,1.92,0.00
4,2024-06-04,1,0,2024-08-11,ES,2,2,0,ES,2.27,0.00


In [16]:
items_minus_df = (
    merged_df
    .groupby(['date', 'country_code'], as_index=False)
    .agg({'items_minus': 'sum'})
)

print(items_minus_df.shape)
items_minus_df.head()

(2639, 3)


,date,country_code,items_minus
0,2024-05-27,AT,0.00
1,2024-05-27,BE,0.00
2,2024-05-27,BG,221.45
3,2024-05-27,CH,0.00
4,2024-05-27,CY,10.80


### Step 4

Using “items minus”, calculate the distribution of exported items for each created_at and aggregate on exported_at 
-> results in dataframe “items_plus”

To find out the distribution of exported items, we need to merge `item_minus_df` with `it_cd_shift_df`. In `item_minus_df` we have date value as created date, but in `it_cd_shift_df` we have export date (`exported_at_picking_warehouse`). We can't directly merge them on this column.

For this, I have created a column called `created_at_estimated` by subtracting delay (`day`) from `exported_at_picking_warehouse` column. Now we can merge the two dataframes. The logic is that the export is done with a delay after created_at date.

* `created_at + day = exported_at`

* hence, `created_at = exported_at - day`

After merging, we can calculate items_plus value using `share` column.

In [17]:
# Let's look at the item cross dock shift dataframe which we will use in this step. 
# This table has the distribution information. 
it_cd_shift_df.head()

,exported_at_picking_warehouse,picking_warehouse_id,shipping_warehouse_id,day,share
0,2024-05-27,1,2,0,0.02
1,2024-05-27,1,2,1,0.23
2,2024-05-27,1,2,2,0.61
3,2024-05-27,1,2,3,0.14
4,2024-05-27,1,2,4,0.00


In [18]:
# Create created_at_estimated column by subtracting day value from exported_at values.
it_cd_shift_df['created_at_estimated'] = it_cd_shift_df['exported_at_picking_warehouse'] - pd.to_timedelta(it_cd_shift_df['day'], unit='D')
print(it_cd_shift_df.shape)
it_cd_shift_df.head()

(5544, 6)


,exported_at_picking_warehouse,picking_warehouse_id,shipping_warehouse_id,day,share,created_at_estimated
0,2024-05-27,1,2,0,0.02,2024-05-27
1,2024-05-27,1,2,1,0.23,2024-05-26
2,2024-05-27,1,2,2,0.61,2024-05-25
3,2024-05-27,1,2,3,0.14,2024-05-24
4,2024-05-27,1,2,4,0.00,2024-05-23


In [19]:
# Now, items_minus_df can be merged with it_cd_shift_df
merged_items_plus = pd.merge(
    items_minus_df,
    it_cd_shift_df,
    left_on='date',
    right_on='created_at_estimated',
    how='inner'
)

print(merged_items_plus.shape)
merged_items_plus.head()

(110838, 9)


,date,country_code,items_minus,exported_at_picking_warehouse,picking_warehouse_id,shipping_warehouse_id,day,share,created_at_estimated
0,2024-05-27,AT,0.0,2024-05-27,1,2,0,0.02,2024-05-27
1,2024-05-27,AT,0.0,2024-05-27,1,3,0,0.01,2024-05-27
2,2024-05-27,AT,0.0,2024-05-27,2,1,0,0.03,2024-05-27
3,2024-05-27,AT,0.0,2024-05-27,2,3,0,0.01,2024-05-27
4,2024-05-27,AT,0.0,2024-05-27,3,1,0,0.03,2024-05-27


In [20]:
# Calculate the items_plus value
merged_items_plus['items_plus'] = merged_items_plus['items_minus'] * merged_items_plus['share']
print(merged_items_plus.shape)
merged_items_plus.head()

(110838, 10)


,date,country_code,items_minus,exported_at_picking_warehouse,picking_warehouse_id,shipping_warehouse_id,day,share,created_at_estimated,items_plus
0,2024-05-27,AT,0.0,2024-05-27,1,2,0,0.02,2024-05-27,0.0
1,2024-05-27,AT,0.0,2024-05-27,1,3,0,0.01,2024-05-27,0.0
2,2024-05-27,AT,0.0,2024-05-27,2,1,0,0.03,2024-05-27,0.0
3,2024-05-27,AT,0.0,2024-05-27,2,3,0,0.01,2024-05-27,0.0
4,2024-05-27,AT,0.0,2024-05-27,3,1,0,0.03,2024-05-27,0.0


In [21]:
items_plus_df = merged_items_plus.groupby(['exported_at_picking_warehouse', 'country_code'])['items_plus'].sum().reset_index()

print(items_plus_df.shape)
items_plus_df.head()

(2813, 3)


,exported_at_picking_warehouse,country_code,items_plus
0,2024-05-27,AT,0.000
1,2024-05-27,BE,0.000
2,2024-05-27,BG,26.574
3,2024-05-27,CH,0.000
4,2024-05-27,CY,1.296


## Step 5

E.Calculate final output: B2C + SCD - items_minus + items_plus


In this step, we will merge the dataframes we have generated.

First, we'll look at the dataframes required for this step. Then perform the merging steps.

1. sales_forecast_items = b2c_items + scd_items
2. sales_forecast_items = sales_forecast_items - items_minus
3. sales_forecast_items = sales_forecast_items + items_plus

Finally, pivot the data to get it in the form required for this case study

In [22]:
print(b2c_aggregated.shape)
b2c_aggregated.head(2)

(2639, 3)


,exported_at,planning_shop_cluster,b2c_items
0,2024-05-27,AT,0
1,2024-05-27,BE,0


In [23]:
print(scd_aggregated.shape)
scd_aggregated.head(2)

(2639, 3)


,exported_at,planning_shop_cluster,scd_items
0,2024-05-27,AT,0
1,2024-05-27,BE,0


In [24]:
print(items_minus_df.shape)
items_minus_df.head(2)


(2639, 3)


,date,country_code,items_minus
0,2024-05-27,AT,0.0
1,2024-05-27,BE,0.0


In [25]:
print(items_plus_df.shape)
items_plus_df.head(2)

(2813, 3)


,exported_at_picking_warehouse,country_code,items_plus
0,2024-05-27,AT,0.0
1,2024-05-27,BE,0.0


In [26]:
# We will combine the results now.
# Let's begin by combining b2c_aggregated and scd_aggregated
combined_b2c_scd = pd.merge(
    b2c_aggregated,
    scd_aggregated,
    on=['exported_at', 'planning_shop_cluster'],
    how='inner'
).fillna(0)

# Adding the values of b2c_items and scd_items together to get the total items forecast
combined_b2c_scd['total_forecast_items'] = combined_b2c_scd['b2c_items'] + combined_b2c_scd['scd_items']

# Dropping individual columns, they are not required anymore
combined_b2c_scd.drop(columns=['b2c_items', 'scd_items'], inplace=True)

# Renaming the new column for clarity
combined_b2c_scd.rename(columns={'total_forecast_items': 'sales_forecast_items'}, inplace=True)

print(combined_b2c_scd.shape)
combined_b2c_scd.head()

(2639, 3)


,exported_at,planning_shop_cluster,sales_forecast_items
0,2024-05-27,AT,0
1,2024-05-27,BE,0
2,2024-05-27,BG,935
3,2024-05-27,CH,0
4,2024-05-27,CY,48


In [27]:
# Now begin by merging the result of last combination dataframe with items_minus.
final_merged = pd.merge(
    combined_b2c_scd,
    items_minus_df,
    left_on=['planning_shop_cluster', 'exported_at'],
    right_on=['country_code', 'date'],
    how='inner'
).fillna(0)

# Subtracting items_minus from sales_forecast_items
final_merged['sales_forecast_items'] = final_merged['sales_forecast_items'] - final_merged['items_minus']

# Dropping the items_minus column
final_merged.drop(columns=['items_minus', 'date', 'planning_shop_cluster'], inplace=True)

print(final_merged.shape)
final_merged.head()

(2639, 3)


,exported_at,sales_forecast_items,country_code
0,2024-05-27,0.00,AT
1,2024-05-27,0.00,BE
2,2024-05-27,713.55,BG
3,2024-05-27,0.00,CH
4,2024-05-27,37.20,CY


In [28]:
# In the last step add the items_plus values to the dataframe we have got until now
final_output = pd.merge(
    final_merged,
    items_plus_df,
    left_on=['country_code', 'exported_at'],
    right_on=['country_code', 'exported_at_picking_warehouse'],
    how='inner'
).fillna(0)

# Adding distributed_items to sales_forecast_items
final_output['sales_forecast_items'] = final_output['sales_forecast_items'] + final_output['items_plus']

# Drop irrelevant columns
final_output.drop(columns=['items_plus','exported_at_picking_warehouse'], inplace=True)

print(final_output.shape)
final_output.head()

(2639, 3)


,exported_at,sales_forecast_items,country_code
0,2024-05-27,0.000,AT
1,2024-05-27,0.000,BE
2,2024-05-27,740.124,BG
3,2024-05-27,0.000,CH
4,2024-05-27,38.496,CY


In [29]:
final_output = final_output.pivot(
    index='country_code', 
    columns='exported_at', 
    values='sales_forecast_items'
)

# Fill missing values. Replace NaN with 0
final_output = final_output.fillna(0)

final_output

exported_at,2024-05-27,2024-05-28,2024-05-29,2024-05-30,2024-05-31,2024-06-01,2024-06-02,2024-06-03,2024-06-04,2024-06-05,...,2024-08-16,2024-08-17,2024-08-18,2024-08-19,2024-08-20,2024-08-21,2024-08-22,2024-08-23,2024-08-24,2024-08-25
country_code,,,,,,,,,,,,,,,,,,,,,
AT,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
BE,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
BG,740.1240,1372.9740,2513.4900,3273.9160,3345.0780,3573.3670,3797.5020,3452.8460,3116.6150,3249.1630,...,5047.6480,5514.7820,7119.7240,5800.5700,4572.4550,3070.4290,3226.8200,3204.0300,3291.7500,3343.2500
CH,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
CY,38.4960,78.7360,115.3040,131.1280,132.2960,132.3120,141.3760,114.2720,115.7600,120.8320,...,160.8880,174.7920,220.1600,149.0640,139.9040,100.2960,99.3760,93.1120,99.1040,97.6720
CZ,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
DE,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
DK,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
EE,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000


We finally get the dataframe in the required format. it gives the items exported at warehouse 2 by country and export dates.